In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('juliet_data.csv')

In [3]:
df

,Unnamed: 0,testcase_ID,filename,code,flaw,flaw_loc,bug
0,0,61940,000/061/940/CWE114_Process_Control__w32_char_c...,/* TEMPLATE GENERATED TESTCASE FILE\nFilename:...,CWE-114,121,False
1,1,61941,000/061/941/CWE114_Process_Control__w32_char_c...,/* TEMPLATE GENERATED TESTCASE FILE\nFilename:...,CWE-114,124,False
2,2,61942,000/061/942/CWE114_Process_Control__w32_char_c...,/* TEMPLATE GENERATED TESTCASE FILE\nFilename:...,CWE-114,124,False
3,3,61943,000/061/943/CWE114_Process_Control__w32_char_c...,/* TEMPLATE GENERATED TESTCASE FILE\nFilename:...,CWE-114,131,False
4,4,61944,000/061/944/CWE114_Process_Control__w32_char_c...,/* TEMPLATE GENERATED TESTCASE FILE\nFilename:...,CWE-114,131,False
...,...,...,...,...,...,...,...
210361,105178,-248721,000/248/721/CWE78_OS_Command_Injection__wchar_...,/* TEMPLATE GENERATED TESTCASE FILE\nFilename:...,CWE-078,137,True
210362,105179,-248722,000/248/722/CWE78_OS_Command_Injection__wchar_...,/* TEMPLATE GENERATED TESTCASE FILE\nFilename:...,CWE-078,137,True
210363,105180,-248722,000/248/722/CWE78_OS_Command_Injection__wchar_...,/* TEMPLATE GENERATED TESTCASE FILE\nFilename:...,CWE-078,137,True
210364,105181,-248722,000/248/722/CWE78_OS_Command_Injection__wchar_...,/* TEMPLATE GENERATED TESTCASE FILE\nFilename:...,CWE-078,137,True


In [4]:
shuffled_df = df.sample(frac=1, random_state=42)

In [5]:
new_df=true_label_df = df[df['bug'] == True].copy()
new_df.reset_index(drop=True, inplace=True)

In [6]:
new_df = new_df[:1000]

In [7]:
import torch
from transformers import BertTokenizer,BertLMHeadModel,BertForMaskedLM

In [8]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',pad_token="[PAD]")
model = BertForMaskedLM.from_pretrained("bert-base-uncased")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [9]:
import re

def remove_comments(code):
    # Regular expression to match C and C++ comments (single-line and multi-line)
    pattern = r'/\*.*?\*/|//.*?$'

    # Remove comments using regex
    cleaned_code = re.sub(pattern, '', code, flags=re.MULTILINE | re.DOTALL)

    return cleaned_code

In [10]:
remove_comments(new_df["code"][0])

'\n\n\n#include "std_testcase.h"\n\n#include <wchar.h>\n\n#ifdef _WIN32\n#include <winsock2.h>\n#include <windows.h>\n#include <direct.h>\n#pragma comment(lib, "ws2_32") \n#define CLOSE_SOCKET closesocket\n#else \n#include <sys/types.h>\n#include <sys/socket.h>\n#include <netinet/in.h>\n#include <arpa/inet.h>\n#include <unistd.h>\n#define INVALID_SOCKET -1\n#define SOCKET_ERROR -1\n#define CLOSE_SOCKET close\n#define SOCKET int\n#endif\n\n#define TCP_PORT 27015\n#define IP_ADDRESS "127.0.0.1"\n\n\n#ifndef OMITBAD\n\nvoid CWE114_Process_Control__w32_char_connect_socket_01_bad()\n{\n    char * data;\n    char dataBuffer[100] = "";\n    data = dataBuffer;\n    {\n#ifdef _WIN32\n        WSADATA wsaData;\n        int wsaDataInit = 0;\n#endif\n        int recvResult;\n        struct sockaddr_in service;\n        char *replace;\n        SOCKET connectSocket = INVALID_SOCKET;\n        size_t dataLen = strlen(data);\n        do\n        {\n#ifdef _WIN32\n            if (WSAStartup(MAKEWORD(2,2)

In [11]:
len(new_df["code"][0])

4344

In [13]:
max_seq_length = 256
code = new_df["code"][0]
code_chunks = [code[i:i + max_seq_length] for i in range(0, len(code), max_seq_length)]

for chunk in code_chunks:
    inputs = tokenizer(chunk, return_tensors='pt', padding=True, truncation=True)
    outputs = model(**inputs)

In [14]:
inputs.input_ids

tensor([[  101,  1013,  1008,  6534,  6721,  2791,  1008,  1013,  5034,  5685,
          1006,  1006, 27121,  1007,  2051,  1006, 19701,  1007,  1007,  1025,
          1001,  2065, 13629,  2546, 18168,  4183,  9024,  6140,  4179,  1006,
          1000,  4214,  2919,  1006,  1007,  1012,  1012,  1012,  1000,  1007,
          1025, 19296,  2063, 14526,  2549,  1035,  2832,  1035,  2491,  1035,
          1035,  1059, 16703,  1035, 25869,  1035,  7532,  1035, 22278,  1035,
          5890,  1035,  2919,  1006,  1007,  1025,  6140,  4179,  1006,  1000,
          2736,  2919,  1006,  1007,  1000,  1007,  1025,  1001,  2203, 10128,
          1013,  1008, 18168,  4183,  9024,  1008,  1013,  2709,  1014,  1025,
          1065,  1001,  2203, 10128,   102]])

In [15]:
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup,TrainingArguments,Trainer
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import re

In [16]:
class VulnerabilityDataset(Dataset):
    def __init__(self, data, max_seq_length=128, max_token_length=1024):
        self.data = data
        self.tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
        self.max_seq_length = max_seq_length
        self.max_token_length = max_token_length

    def remove_comments(self, code):
        # Regular expression to match C and C++ comments (single-line and multi-line)
        pattern = r'/\*.*?\*/|//.*?$'

        # Remove comments using regex
        cleaned_code = re.sub(pattern, '', code, flags=re.MULTILINE | re.DOTALL)

        return cleaned_code

    def tokenize_long_code(self, long_code):
        # Split the long code into smaller segments based on max_token_length
        segments = [long_code[i:i+self.max_token_length] for i in range(0, len(long_code), self.max_token_length)]

        # Tokenize each segment separately and pad them to the same length
        tokenized_segments = []
        max_length = 0
        for segment in segments:
            tokens = self.tokenizer.encode(segment, return_tensors="pt", truncation=True)
            tokenized_segments.append(tokens)
            max_length = max(max_length, tokens.size(1))

        # Pad the segments to the same length
        padded_segments = []
        for tokens in tokenized_segments:
            padded_tokens = torch.cat([tokens, torch.zeros(1, max_length - tokens.size(1), dtype=torch.long)], dim=1)
            padded_segments.append(padded_tokens)

        return padded_segments

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        code = row["code"]
        # cwe_id_str = row["flaw"]  # Assuming it's in the format "cwe-XXX"
        
        # Extract the integer part of the CWE ID
        # cwe_id = int(cwe_id_str.split('-')[-1])

        # Remove comments from the code
        cleaned_code = self.remove_comments(code)

        # Tokenize the cleaned code, including handling long code
        tokenized_code_segments = self.tokenize_long_code(cleaned_code)

        # Combine and pad code segments to create a single input
        combined_input = torch.cat(tokenized_code_segments, dim=1)

        # Trim or pad the sequence to the specified max_seq_length
        if combined_input.size(1) > self.max_seq_length:
            combined_input = combined_input[:, :self.max_seq_length]
        else:
            padding = torch.zeros(1, self.max_seq_length - combined_input.size(1), dtype=torch.long)
            combined_input = torch.cat([combined_input, padding], dim=1)

        return {
            "input_ids": combined_input.squeeze(),
            "flaw": cwe_id,  # Now cwe_id is an integer
        }

In [17]:
new_df.columns

Index(['Unnamed: 0', 'testcase_ID', 'filename', 'code', 'flaw', 'flaw_loc',
       'bug'],
      dtype='object')

In [18]:
data = new_df.drop(["Unnamed: 0","testcase_ID","filename","flaw_loc","bug","flaw"],axis=1)

In [19]:
data.describe()

,code
count,1000
unique,1000
top,/* TEMPLATE GENERATED TESTCASE FILE\nFilename:...
freq,1


In [20]:
num_epochs = 10
batch_size = 16
learning_rate = 5e-5

In [21]:
train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)

In [22]:
train_dataset = VulnerabilityDataset(train_data)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [23]:
val_dataset = VulnerabilityDataset(val_data)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

In [24]:
optimizer = AdamW(model.parameters(), lr=learning_rate)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(train_loader) * num_epochs)
loss_fn = torch.nn.CrossEntropyLoss()

c:\Users\prate\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [25]:
training_args = TrainingArguments(
    per_device_train_batch_size=batch_size,  # Adjust the batch size as needed
    output_dir="./bert_fine_tuned_model_b16",
    num_train_epochs=num_epochs,  # Adjust the number of epochs as needed
    evaluation_strategy="steps",
    eval_steps=100,
    save_steps=100,
    logging_steps=10,
    save_total_limit=2,
    remove_unused_columns=False,
)

In [26]:
model = BertForMaskedLM.from_pretrained("bert-base-uncased")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [28]:
from transformers import DataCollatorForLanguageModeling
class CustomDataCollator(DataCollatorForLanguageModeling):
    def __init__(self, tokenizer, model):
        super().__init__(
            tokenizer=tokenizer,
            mlm=True,
            mlm_probability=0.15,
        )
        self.model = model

    def collate_batch(self, batch):
        input_ids = torch.stack([example["input_ids"] for example in batch])
        attention_mask = torch.stack([example["attention_mask"] for example in batch])
        labels = torch.stack([example["input_ids"] for example in batch])  # Self-prediction task: labels are the same as input

        inputs = {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": labels,
        }
        return inputs

In [29]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=CustomDataCollator(tokenizer, model), 
    train_dataset=train_dataset,
    eval_dataset = val_dataset,# Use the DataLoader directly
    compute_metrics=None,
)

In [30]:
trainer.train()

  0%|          | 0/500 [00:00<?, ?it/s]

{'loss': 2.7883, 'learning_rate': 4.9e-05, 'epoch': 0.2}
{'loss': 1.5065, 'learning_rate': 4.8e-05, 'epoch': 0.4}
{'loss': 0.9807, 'learning_rate': 4.7e-05, 'epoch': 0.6}
{'loss': 0.7234, 'learning_rate': 4.600000000000001e-05, 'epoch': 0.8}
{'loss': 0.5654, 'learning_rate': 4.5e-05, 'epoch': 1.0}
{'loss': 0.5836, 'learning_rate': 4.4000000000000006e-05, 'epoch': 1.2}
{'loss': 0.46, 'learning_rate': 4.3e-05, 'epoch': 1.4}
{'loss': 0.4231, 'learning_rate': 4.2e-05, 'epoch': 1.6}
{'loss': 0.3406, 'learning_rate': 4.1e-05, 'epoch': 1.8}
{'loss': 0.3826, 'learning_rate': 4e-05, 'epoch': 2.0}


  0%|          | 0/25 [00:00<?, ?it/s]

{'eval_loss': 0.22164955735206604, 'eval_runtime': 56.4899, 'eval_samples_per_second': 3.54, 'eval_steps_per_second': 0.443, 'epoch': 2.0}
{'loss': 0.2957, 'learning_rate': 3.9000000000000006e-05, 'epoch': 2.2}
{'loss': 0.3453, 'learning_rate': 3.8e-05, 'epoch': 2.4}
{'loss': 0.2422, 'learning_rate': 3.7e-05, 'epoch': 2.6}
{'loss': 0.2112, 'learning_rate': 3.6e-05, 'epoch': 2.8}
{'loss': 0.228, 'learning_rate': 3.5e-05, 'epoch': 3.0}
{'loss': 0.2373, 'learning_rate': 3.4000000000000007e-05, 'epoch': 3.2}
{'loss': 0.1817, 'learning_rate': 3.3e-05, 'epoch': 3.4}
{'loss': 0.1659, 'learning_rate': 3.2000000000000005e-05, 'epoch': 3.6}
{'loss': 0.1995, 'learning_rate': 3.1e-05, 'epoch': 3.8}
{'loss': 0.1825, 'learning_rate': 3e-05, 'epoch': 4.0}


  0%|          | 0/25 [00:00<?, ?it/s]

...

  0%|          | 0/25 [00:00<?, ?it/s]

{'eval_loss': 0.07428160309791565, 'eval_runtime': 61.2137, 'eval_samples_per_second': 3.267, 'eval_steps_per_second': 0.408, 'epoch': 8.0}
{'loss': 0.0768, 'learning_rate': 9e-06, 'epoch': 8.2}
{'loss': 0.0903, 'learning_rate': 8.000000000000001e-06, 'epoch': 8.4}
{'loss': 0.0836, 'learning_rate': 7.000000000000001e-06, 'epoch': 8.6}
{'loss': 0.0589, 'learning_rate': 6e-06, 'epoch': 8.8}
{'loss': 0.0993, 'learning_rate': 5e-06, 'epoch': 9.0}
{'loss': 0.0662, 'learning_rate': 4.000000000000001e-06, 'epoch': 9.2}
{'loss': 0.1017, 'learning_rate': 3e-06, 'epoch': 9.4}
{'loss': 0.0875, 'learning_rate': 2.0000000000000003e-06, 'epoch': 9.6}
{'loss': 0.0976, 'learning_rate': 1.0000000000000002e-06, 'epoch': 9.8}
{'loss': 0.1108, 'learning_rate': 0.0, 'epoch': 10.0}


  0%|          | 0/25 [00:00<?, ?it/s]

{'eval_loss': 0.05617446452379227, 'eval_runtime': 61.3213, 'eval_samples_per_second': 3.262, 'eval_steps_per_second': 0.408, 'epoch': 10.0}
{'train_runtime': 7684.9701, 'train_samples_per_second': 1.041, 'train_steps_per_second': 0.065, 'train_loss': 0.2883510800600052, 'epoch': 10.0}


TrainOutput(global_step=500, training_loss=0.2883510800600052, metrics={'train_runtime': 7684.9701, 'train_samples_per_second': 1.041, 'train_steps_per_second': 0.065, 'train_loss': 0.2883510800600052, 'epoch': 10.0})